# Imports, Definitions, and Instantiating the overlay

In [1]:
# from mkidgen3.testutils import *
# import mkidgen3.testutils as tu
# from mkidgen3.fixedpoint import *
import logging
# import itertools
# import scipy.signal
# import os
# from glob import glob

import pynq
import time
from pynq import PL
import xrfclk
import xrfdc
import numpy as np
from fpbinary import FpBinary
import matplotlib.pyplot as plt
from scipy.fftpack import fft, fftfreq, fftshift
import mkidgen3 as g3
import mkidgen3.mkidpynq as mkidpynq
import mkidgen3.daccomb as daccomb
import mkidgen3
from mkidgen3.daccomb import generate_dac_comb
from mkidgen3.drivers import axiswitch, bintores, capture, ddc, dactable, axififo, rfdc, phasematch, iqgen

FP16_23 = lambda x: FpBinary(int_bits=-7, frac_bits=23, signed=True, value=x)
FP16_15 = lambda x: FpBinary(int_bits=1, frac_bits=15, signed=True, value=x)
FP16_14 = lambda x: FpBinary(int_bits=2, frac_bits=14, signed=True, value=x)
FP16_13 = lambda x: FpBinary(int_bits=3, frac_bits=13, signed=True, value=x)
FP16_10 = lambda x: FpBinary(int_bits=6, frac_bits=10, signed=True, value=x)
FP16_16 = lambda x: FpBinary(int_bits=0, frac_bits=16, signed=True, value=x)
FP16_14bf = lambda x: FpBinary(int_bits=2, frac_bits=14, signed=True, bit_field=x)
FP16_16bf = lambda x: FpBinary(int_bits=0, frac_bits=16, signed=True, bit_field=x)
FP16_10bf = lambda x: FpBinary(int_bits=6, frac_bits=10, signed=True, bit_field=x)
FP16_8 = lambda x: FpBinary(int_bits=8, frac_bits=8, signed=True, value=x)
FP16_8bf = lambda x: FpBinary(int_bits=8, frac_bits=8, signed=True, bit_field=x)

n_res = 2048
n_bin = 4096
matched_filter_loaded=False
logging.basicConfig()
logging.getLogger('').setLevel('INFO')
logging.getLogger('__main__').setLevel('INFO')

In [2]:
ol = g3.configure('zcu111a/test_phasematch.bit', clocks=True, external_10mhz=True, ignore_version=True)
print(f"PL Bitfile: {PL.bitfile_name}\nPL Timestamp: {PL.timestamp}\n"
      f"Overlay timestamp: {ol.timestamp}  Loaded: {ol.is_loaded()}")

PL Bitfile: /home/xilinx/jupyter_notebooks/test_phasematch/zcu111a/test_phasematch.bit
PL Timestamp: 2022/3/3 0:24:24 +811665
Overlay timestamp: 2022/3/3 0:24:24 +811665  Loaded: True


In [3]:
class PhaseFIRCoeffFile:
    def __init__(self, file):
        self.file=file
        npz = np.load(self.file)
        self.coeffs = npz['filters']
        
class Foo:
    def set_driver(self,*args,**kw):
        return
ol.capture.switch=Foo()

In [4]:
ol.capture, ol.phasematch, ol.iq_gen_0.register_map

(<mkidgen3.drivers.capture.CaptureHierarchy at 0xffff80277d00>,
 RegisterMap {
   max = Register(max=0, RESERVED=0),
   run = Register(run=0, RESERVED=0)
 })

In [5]:
ol.iq_gen_0.register_map

RegisterMap {
  max = Register(max=0, RESERVED=0),
  run = Register(run=0, RESERVED=0)
}

In [6]:
#ol.iq_gen_0.register_map.max=4096 #2**20-1
ol.iq_gen_0.register_map.max=2**20-1
ol.iq_gen_0.register_map.max

Register(max=1048575, RESERVED=0)

In [7]:
ol.iq_gen_0.register_map.run=True

## Test the basics of the driver

In [ ]:
ncoeff=30
res_id=4  #lane 0 coeff set 1
lane = res_id % phasematch.PhasematchDriver.N_LANES
reload_packet = np.zeros(phasematch.PhasematchDriver.N_TEMPLATE_TAPS + 1, dtype=np.uint16)
reload_packet[0] = res_id // phasematch.PhasematchDriver.N_LANES
# reload_packet[1:] = phasematch.PhasematchDriver.reorder_coeffs(2+np.arange(phasematch.PhasematchDriver.N_TEMPLATE_TAPS))]
reload_packet[1:] = 1+np.arange(phasematch.PhasematchDriver.N_TEMPLATE_TAPS)
cfg_packet = np.arange(phasematch.PhasematchDriver.N_RES_P_LANE, dtype=np.uint16)
reload_packet = mkidpynq.pack16_to_32(reload_packet)
cfg_packet = mkidpynq.pack16_to_32(cfg_packet)
reload_packet,lane,reload_packet[0], cfg_packet

In [ ]:
ol.phasematch.fifo.tx(mkidpynq.pack16_to_32(reload_packet), destination=lane, last_bytes=2)

In [ ]:
ol.phasematch.fifo.tx(cfg_packet, destination=4)  # Send a config packet to trigger the reload

In [8]:
cfg_packet = np.arange(phasematch.PhasematchDriver.N_RES_P_LANE, dtype=np.uint16)
cfg_packet = mkidpynq.pack16_to_32(cfg_packet)
reload_packet = np.zeros(phasematch.PhasematchDriver.N_TEMPLATE_TAPS + 1, dtype=np.uint16)

def prog(res_id):
    lane = res_id % phasematch.PhasematchDriver.N_LANES
    reload_packet[0] = res_id // phasematch.PhasematchDriver.N_LANES
    reload_packet[1:]=0
    if res_id in (0,3):
        reload_packet[1]=0x7fff
    elif res_id in (4, 6,7):
         reload_packet[1]=0x7fff//2
    elif res_id in (2046,2045,244,2047,):
        reload_packet[1]=0x7fff*3//4
    reload_packet[1:] = reload_packet[1:][::-1]
    ol.phasematch.fifo.tx(mkidpynq.pack16_to_32(reload_packet), destination=lane, last_bytes=2)
    if (res_id>0 and res_id%8==3):
        ol.phasematch.fifo.tx(cfg_packet, destination=4) 

In [11]:
for res_id in range(2048):
    prog(res_id)
toc=time.time()
print(toc-tic)

ValueError: Insufficient room in fifo for data

In [ ]:
for i in range(1024,1031): prog(i)

In [ ]:
prog(1031)

In [ ]:
ol.phasematch.fifo.tx(cfg_packet, destination=4) 

In [10]:
ol.phasematch.stream_gate_0.register_map.run=True

In [ ]:
[#,X,X,#, X,#,X,#, X,...,X, X,X,X,#]

In [ ]:
##..#.#.#.......

In [ ]:
try:
    phase.freebuffer()
except:
    pass
phase = ol.capture.capture_phase(8192*2, groups=('all'), duration=False)
ps=np.array(phase)
p=np.array(phase).ravel()
g0=np.argwhere(p>0).ravel()
f=g0[0]
p[f:f+20], p[f+2020:f+2060]

In [ ]:
np.argwhere((ps!=0).sum(0))

In [ ]:
(np.argwhere((ps!=0).sum(0))-1108+244+2048)%2048

In [ ]:
(g0-f)[:100]

lanes are coming in 1,2,3,4...
lane channels are seeing i * 512 + lane channel, i from 0-127 
lane channel is resonator channel/4
resonator r is in lane%4 and sees i * 512 + r//4

In [ ]:
(g0-f)[:10]

In [ ]:
(p[600:1200])

In [ ]:
each capture of 16 phases has 4 of each lane, I'd expect data order to be l0l1l2l3l0l1l2l3... but it seems to be l0l0l0l0 ....

In [ ]:
p[500:1000]/2**8

In [ ]:
(FP16_14bf(12365+64*30)*FP16_15(.99998)).resize((8,8)).__index__()

In [ ]:
bin(13824), hex(phasematch.FP16_15(.99998).__index__())

# Now for a real test

## Prepare the Data

In [ ]:
n_total_packets=301  # How many packets
sample_rate=2.048e6
PULSE_DECAY_TIME=15e-6
pulse_times=np.linspace(PULSE_DECAY_TIME,4*5*PULSE_DECAY_TIME, 5)
amplitude = 1

Turn one ideal pulse into phase. Take 30 points, time reverse it, and normalize it so that the maximum of a filtered pulse gives the original pulse height.

In [ ]:
def pulse(t, decay):
    heavy_e=-np.e**(-t/PULSE_DECAY_TIME)*np.heaviside(t,1)/2
    return heavy_e/2 + heavy_e*1j

def gen_matched_filt(n_taps=30, n_before=2, plot=True, sample_rate=1.024e6):
    t=np.arange(n_taps)/sample_rate
    x=pulse(t-n_before*np.diff(t)[0], PULSE_DECAY_TIME)+1
    phase=np.arctan2(x.imag, x.real)/np.pi
    
    matched = phase[::-1]
    norm=np.sign(phase[np.abs(phase).argmax()])*np.abs(phase).max()/np.abs(scipy.signal.convolve(phase, matched,'same')).max()
    if plot:
        plt.subplot(1,2,1)
        plt.plot(x.real)
        plt.plot(x.imag)
        plt.xlabel('Sample')
        plt.ylabel('I and Q')
        plt.subplot(1,2,2)
        plt.plot(phase, label='phase')
        plt.plot(matched, label='filt')
        plt.plot(scipy.signal.convolve(phase, matched*norm,'same'),label='convol')
        plt.xlabel('Sample')
        plt.ylabel('Phase')
        plt.legend()
        plt.tight_layout()
    return matched*norm

Generate the complex waveform for a single resonator.

In [ ]:
%matplotlib inline
max_t=1.25*(PULSE_DECAY_TIME+pulse_times.max())  #How many waveform samples do we need to generate
n_samples=int(np.ceil(max_t*sample_rate))
t = np.arange(n_samples)/sample_rate

comb=np.ones_like(t, dtype=np.complex64)
for t0 in pulse_times:
    comb+=pulse(t-t0, PULSE_DECAY_TIME)
comb*=amplitude

print(f"Comb shape: {comb.shape}. \nTotal Samples: {comb.size}. Memory: {comb.size*8/1024**2:.0f} MB\n"
      f"Max value: {np.abs(comb).max()}.\n"
      f"Expected tone amplitude factor: ~512 * N_TONES_IN_BIN. (4096/8 as FFT scale dfaults to 8 in last 3 stages)\n"
      f"Resulting samples per output bin: {comb.size*2/n_bin}")

plt.tight_layout()
plt.figure()
plt.subplot(1,2,1)
plt.plot(comb.real,comb.imag)
plt.xlabel('I')
plt.ylabel('Q')
plt.subplot(1,2,2)
plt.plot(t*1e6, comb.real)
plt.plot(t*1e6, comb.imag)
plt.ylabel('IQ Amplitude')
plt.xlabel('t (us)')
plt.tight_layout()

### Configure the phase FIRs

Make the matched filter

In [ ]:
%matplotlib inline
matched_filt=gen_matched_filt(n_taps=30,n_before=2, plot=True)

Load the coeffs

In [ ]:
for i in range(512):
    getLogger(__name__).info(f'Loading coefficient set {i}')
    for j in range(4):  # We can load one set in each FIR bank before needing to trigger a config event.
        res_id = j*512 + i
        data = matched_filt
#         data=np.zeros_like(matched_filt, dtype=np.uint16)
#         data[-1]=0x7fff
        ol.phasematch.load_coeff(res_id, data)
    # Send zeros to trigger a config event, the config block will ensure a config packet is ready
    drive_data(dma,0, zeros=True, in_per_out=2, n_out=1, phaseout=True, n_latency_packets=1)
matched_filter_loaded=True

### Configure the DDS

We are going to feed in tones at baseband, so the increment and phase offset are 0.

In [ ]:
tones=np.zeros((2,2048))
ddc.tones=tones

## Manually test a single path

res_id=3
coeffs=np.arange(30)
lane = res_id // PhasematchDriver.N_RES_P_LANE
reload_packet = np.zeros(coeffs.size + 1, dtype=np.uint16)
reload_packet[0] = res_id % PhasematchDriver.N_RES_P_LANE
FP32_8 = lambda x: FpBinary(int_bits=32 - 9, frac_bits=8, signed=True, value=x)
reload_packet[1:] = [FP32_8(c).__index__() for c in PhasematchDriver.reorder_coeffs(coeffs)]

cfg_packet=np.arange(PhasematchDriver.N_RES_P_LANE, dtype=np.uint16)

reload_packet=pack16_to_32(reload_packet)
cfg_packet=pack16_to_32(cfg_packet)

fifo.tx(reload_packet, destination=0, last_bytes=2)  #reload channels are 0-3
pm.fifo.register_map
fifo.tx(reload_packet, destination=1, last_bytes=2)
pm.fifo.register_map

ol.phasematch.load_coeff(res_id, np.arange(30))

fifo.register_map.SRR=0x000000A5
fifo.reset_tx_fifo()

# Generate and feed a stream

First create a timeseries of IQ values for a DDSed resonator.

In [ ]:
n_out=4
out=[]
n_latency_packets=1
n_sent=0
while n_sent+2*n_out<comb.size:
    # Send zeros to trigger a config event, the config block will ensure a config packet is ready  
    out.append(drive_data(dma, comb[n_sent:], in_per_out=2, n_out=n_out, phaseout=True, fpgen=FP16_15, 
                          fprecv=None, n_latency_packets=n_latency_packets)[::2048])
    n_sent+=n_latency_packets*(n_sent==0)+2*n_out    
out=np.concatenate(out)

In [ ]:
outfp=np.array([float(FpBinary(8,8,True,bit_field=int(x))) for x in out])
#outfp2=np.array([float(FpBinary(9,7,True,bit_field=int(x))) for x in out]) #Use this line if the plots at the end aren't matching

## Simulate the results

First define a bunch of helpers

In [ ]:
with open(f"data/fclowpass.coe",'r') as f:
    lines=f.readlines()[1:]
lines[0]=lines[0].partition('=')[-1]
lpcoeffs=np.array(list(map(float,''.join(lines).replace(';','').replace('\n','').split(','))))  #c19-0

In [ ]:
#Lowpass
lowpassed = np.zeros(comb.size-lpcoeffs.size+1, dtype=np.complex64)
lowpassed.real = np.convolve(comb.real, lpcoeffs[::-1], mode='valid')
lowpassed.imag = np.convolve(comb.imag, lpcoeffs[::-1], mode='valid')
lowpassed=scipy.signal.decimate(lowpassed,2, n=lpcoeffs.size-1, ftype='fir', zero_phase=False)

#arctan the IQ
phased = np.arctan2(lowpassed.imag, lowpassed.real)/np.pi

#Filter the phases
matched = np.convolve(phased, matched_filt, mode='valid')

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(2,2, figsize=(12,8), sharex=True)
plt.sca(ax.flat[0])
plt.plot(t*1e6, comb.real)
plt.plot(t*1e6, comb.imag)
plt.plot(t*1e6, np.abs(comb))
plt.ylabel('I & Q Samples w/gain')

plt.sca(ax.flat[1])
plt.plot(t[:lowpassed.size*2:2]*1e6,np.abs(lowpassed))
plt.ylabel('|IQ| (lowpass applied)')
plt.xlabel('t (us)')

plt.sca(ax.flat[2])
plt.plot(t*1e6,np.arctan2(comb.imag,comb.real)/np.pi)
plt.ylabel('atan2(I/Q)/pi')


plt.sca(ax.flat[3])
plt.plot(t[:phased.size*2:2]*1e6,phased)
plt.ylabel('Atan2(I/Q)/pi (after lowpass)')
#plt.xlim(0,1500)
plt.xlabel('t (us)')
plt.tight_layout()

plt.figure()
plt.plot(t[:matched.size*2:2]*1e6,matched, label='Python')
plt.ylabel('Phase (lowpass applied)')
plt.xlabel('t (us)')
#plt.xlim(0,1400)

In [ ]:
clip_ndx=29
clipped_fpga=outfp[clip_ndx:]
fpga_t=t[::2][:clipped_fpga.size]
plt.plot(1e6*fpga_t,clipped_fpga, label='FPGA')

if matched_filter_loaded:
    plt.plot(t[:matched.size*2:2]*1e6,matched, label='Python')
    plt.ylabel('Filtered Phase (lowpass applied)')
else:
    plt.plot(t[:phased.size*2:2]*1e6,-phased, label='Python')
    plt.ylabel('Phase (lowpass applied)')

plt.xlabel('t (us)')
plt.legend()